# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

import sys
print("Python путь:", sys.executable)
print("Должен содержать: /python_projects/venv_langchain/")

Python путь: /Users/sergej/Library/Mobile Documents/com~apple~CloudDocs/DS_AI/python_projects/venv_langchain/bin/python
Должен содержать: /python_projects/venv_langchain/


In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
# !pip install langchain
#!pip install langchain-openai

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [14]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None, model="gpt-5-nano")


In [29]:
prompt = ChatPromptTemplate.from_template("Describe main features of the product: {input_parameter}?")   

In [30]:
chain = LLMChain(llm=llm, prompt=prompt)

In [31]:
product = "Iphone 14 Pro"
chain.run(product)

'Here’s a concise overview of the main features of the iPhone 14 Pro:\n\n- Design and display\n  - 6.1-inch Super Retina XDR OLED with ProMotion (adaptive 120Hz) and Always-On Display\n  - Dynamic Island: the notch area becomes an interactive UI element for alerts, controls, and live activities\n  - Front Ceramic Shield, surgical-grade stainless steel frame, IP68 water/dust resistance\n  - Color options include Space Black, Silver, Gold, and Deep Purple\n\n- Performance\n  - A16 Bionic chip\n  - 6 GB of RAM\n  - Storage options: 128 GB, 256 GB, 512 GB, and 1 TB\n  - 5G connectivity\n\n- Camera system\n  - 48 MP main wide camera with a quad-pixel sensor for high-res stills and improved low-light performance\n  - 12 MP ultra-wide camera\n  - 12 MP telephoto camera with 3x optical zoom\n  - Advanced computational photography features: Photonic Engine, Night mode, Deep Fusion, ProRAW, and ProRes video (high-quality cinema-style formats)\n  - Cinematic mode for video (up to 4K)\n\n- Battery

## SimpleSequentialChain

In [32]:
from langchain.chains import SimpleSequentialChain

In [33]:
llm = ChatOpenAI(temperature=0.9, model="gpt-5-nano")

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Find the closest Android version for the product: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [34]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Describe the product:\n\nOutline:\n{outline}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [35]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [36]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
There isn’t one. The iPhone 14 Pro runs iOS, not Android, so there is no Android version for this device.

If you’re looking for a rough cross-platform reference by release year:
- iPhone 14 Pro launched Sept 2022 with iOS 16 (and can run iOS 17 now).
- The closest Android version released around that time is Android 13 (Aug 2022), with Android 14 following in 2023.

If you want, I can compare iPhone 14 Pro’s features with an Android phone from that era (e.g., Galaxy S22) or summarize iOS vs Android features.
Product description: Apple iPhone 14 Pro

Overview
The iPhone 14 Pro is Apple’s flagship smartphone, built around iOS and designed for high-end performance, advanced photography/video capabilities, and a premium, durable build. It uses Apple’s A16 Bionic chip, a 6.1-inch ProMotion display, and a multifunction Dynamic Island that blends hardware and software interactions. It runs iOS (initially iOS 16, upgradable to iOS 17 and beyond)

'Product description: Apple iPhone 14 Pro\n\nOverview\nThe iPhone 14 Pro is Apple’s flagship smartphone, built around iOS and designed for high-end performance, advanced photography/video capabilities, and a premium, durable build. It uses Apple’s A16 Bionic chip, a 6.1-inch ProMotion display, and a multifunction Dynamic Island that blends hardware and software interactions. It runs iOS (initially iOS 16, upgradable to iOS 17 and beyond) and is focused on a seamless Apple ecosystem experience with strong security features and ongoing software support.\n\nKey features\n- Display: 6.1-inch LTPO OLED with ProMotion (120 Hz) for smooth scrolling and responsive visuals; features an Always-On display and Dynamic Island, which adapts to notifications and app activities.\n- Performance: A16 Bionic processor delivering strong application and camera performance, smooth UI, and energy efficiency.\n- Camera system: Triple-camera setup\n  - 48 MP main sensor with a quad-pixel or large-pixel readout

**Repeat the above twice for different products**

## SequentialChain

In [37]:
from langchain.chains import SequentialChain

In [38]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review to German:\n\nReview: {review}"
)


chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="german_review" # Даем имя выходу
                    )


In [41]:

second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in 1-2 sentences:\n\nReview: {german_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" # give a name to this output
                    )


In [44]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
      "Translate the following review to Russian:\n\nReview: {review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="russian_review"
                      )


In [47]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables

fourth_prompt = ChatPromptTemplate.from_template(
    "Here is the original review in Russian:\n{russian_review}\n\nHere is its summary in German:\n{summary}\n\nWrite a follow-up message to the customer in Russian, addressing their review and summary."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [50]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["summary", "russian_review", "followup_message"],
    verbose=True
)

In [51]:
review = df.Review[5]
overall_chain(review)

/var/folders/hg/7_xt468j3nz1xqspwcc6s8jw0000gn/T/ipykernel_40466/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'summary': 'Der Kunde findet den Geschmack schlecht, bemerkt, dass der Schaum nicht gut hält und vermutet, dass es sich um eine alte Charge oder eine Fälschung handelt im Vergleich zu denen, die er im Laden kauft.',
 'russian_review': 'Отзыв: Я нахожу вкус отвратительным. Пена не держится, это странно. Я покупаю те же самые в магазине, и вкус намного лучше... Старая партия или подделка!?',
 'followup_message': 'Уважаемый клиент,\n\nМы приносим извинения за ваше негативное впечатление от продукции. Мы серьезно относимся к качеству нашей продукции и стремимся обеспечить клиентов только лучшими продуктами.\n\nМы хотели бы уточнить, что наша продукция проходит строгий контроль качества, и мы не используем старые партии или подделки. Возможно, у нас произошла непредвиденная ситуация, и мы готовы рассмотреть вашу пр

**Repeat the above twice for different products or reviews**

## Router Chain

In [52]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [53]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [54]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [55]:
llm = ChatOpenAI(temperature=0)

In [56]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [57]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [58]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [59]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [60]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/hg/7_xt468j3nz1xqspwcc6s8jw0000gn/T/ipykernel_40466/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [61]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities, which depends only on the temperature of the body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is proportional to the frequency of the radiation and the temperature of the body."

In [62]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [63]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**